# 🧠 RNN/LSTM/GRU для временных рядов

**Phase 3: Temporal Data & RNN - Step 2**

---

## 🎯 Цели ноутбука

1. **Понять архитектуры RNN:** Vanilla RNN, LSTM, GRU
2. **Проблемы RNN:** vanishing/exploding gradients
3. **Практика на PyTorch:** реализация для прогнозирования
4. **Сравнение с классикой:** RNN vs ARIMA/SARIMA
5. **Продвинутые техники:** Bidirectional, Stacked RNN

---

## 📊 Датасет: Airline Passengers

**Используем тот же датасет** для честного сравнения с ARIMA/SARIMA/Prophet.

**Зачем Deep Learning?**
- 📈 **Нелинейные паттерны:** RNN могут захватывать сложные зависимости
- 🔗 **Долгосрочная память:** LSTM для long-term dependencies
- 🧠 **Автоматические признаки:** не нужно вручную создавать лаги
- 🎯 **Multivariate готовность:** легко добавить дополнительные признаки

**Честное предупреждение:**
- ❌ **RNN НЕ всегда лучше** классических методов для univariate TS
- ❌ **Требуют больше данных** (наш датасет маленький - 144 точки)
- ❌ **Дольше обучаются** и сложнее настраивать
- ✅ **Цель:** понять основы для более сложных задач

---

## 📚 Часть 1: Теория рекуррентных сетей

### 1.1 Vanilla RNN (Простая рекуррентная сеть)

**Идея:** На каждом временном шаге сеть обрабатывает вход $x_t$ и скрытое состояние $h_{t-1}$.

#### Архитектура

**Forward pass:**

$$h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$$

$$y_t = W_{hy} h_t + b_y$$

где:
- $x_t$ — вход на шаге $t$ (например, количество пассажиров)
- $h_t$ — скрытое состояние (hidden state) на шаге $t$
- $h_{t-1}$ — скрытое состояние с предыдущего шага
- $W_{xh}$ — веса для входа
- $W_{hh}$ — веса для скрытого состояния (рекуррентные)
- $W_{hy}$ — веса для выхода
- $\tanh$ — функция активации

**Визуализация (развернутая во времени):**

```
x₁  →  [RNN]  →  h₁  →  y₁
         ↓
x₂  →  [RNN]  →  h₂  →  y₂
         ↓
x₃  →  [RNN]  →  h₃  →  y₃
         ↓
       ...
```

---

### 1.2 Проблема Vanishing Gradient

**Backpropagation Through Time (BPTT):**

При обратном распространении через много временных шагов:

$$\frac{\partial L}{\partial h_1} = \frac{\partial L}{\partial h_T} \prod_{t=2}^{T} \frac{\partial h_t}{\partial h_{t-1}}$$

Градиент:

$$\frac{\partial h_t}{\partial h_{t-1}} = W_{hh}^T \cdot \text{diag}(1 - \tanh^2(z_t))$$

**Проблема:**
- Если $\|W_{hh}\| < 1$ → градиенты **исчезают** (vanishing)
- Если $\|W_{hh}\| > 1$ → градиенты **взрываются** (exploding)

**Результат:** Vanilla RNN **плохо обучается** на длинных последовательностях (> 10-20 шагов).

**Решение:** LSTM и GRU!

---

### 1.3 LSTM (Long Short-Term Memory)

**LSTM решает проблему vanishing gradient** через механизм **gates** (ворот).

#### Архитектура LSTM

LSTM имеет **два состояния:**
- $c_t$ — **cell state** (долгосрочная память)
- $h_t$ — **hidden state** (краткосрочная память)

**Три gates:**

**1. Forget Gate** (что забыть из cell state):

$$f_t = \sigma(W_f [h_{t-1}, x_t] + b_f)$$

**2. Input Gate** (что добавить в cell state):

$$i_t = \sigma(W_i [h_{t-1}, x_t] + b_i)$$

$$\tilde{c}_t = \tanh(W_c [h_{t-1}, x_t] + b_c)$$

**3. Output Gate** (что вывести):

$$o_t = \sigma(W_o [h_{t-1}, x_t] + b_o)$$

---

**Обновление состояний:**

$$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$$

$$h_t = o_t \odot \tanh(c_t)$$

где:
- $\sigma$ — sigmoid (выход 0-1, работает как gate)
- $\odot$ — element-wise умножение
- $[h_{t-1}, x_t]$ — конкатенация

---

**Интуиция:**
- **Forget gate:** "Сколько забыть из прошлого?"
- **Input gate:** "Сколько новой информации добавить?"
- **Output gate:** "Что вывести на основе памяти?"

**Преимущество:** Cell state $c_t$ течет через всю последовательность с минимальными изменениями → **нет vanishing gradient**!

---

### 1.4 GRU (Gated Recurrent Unit)

**GRU** — упрощенная версия LSTM с **меньшим количеством параметров**.

#### Архитектура GRU

**Два gates:**

**1. Reset Gate** (сбросить прошлую информацию):

$$r_t = \sigma(W_r [h_{t-1}, x_t] + b_r)$$

**2. Update Gate** (обновить состояние):

$$z_t = \sigma(W_z [h_{t-1}, x_t] + b_z)$$

**Кандидат на новое состояние:**

$$\tilde{h}_t = \tanh(W_h [r_t \odot h_{t-1}, x_t] + b_h)$$

**Финальное состояние:**

$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

---

**Отличия от LSTM:**
- ✅ **Меньше параметров** (2 gates vs 3)
- ✅ **Быстрее обучается**
- ✅ **Одно состояние** ($h_t$ вместо $h_t$ и $c_t$)
- ⚠️ **Производительность:** обычно сопоставима с LSTM, иногда чуть хуже

---

### 1.5 Сравнение: RNN vs LSTM vs GRU

| Характеристика | Vanilla RNN | LSTM | GRU |
|----------------|-------------|------|-----|
| **Параметры** | Мало | Много (4x) | Средне (3x) |
| **Скорость** | Быстро | Медленно | Средне |
| **Long-term dependencies** | ❌ Плохо | ✅ Отлично | ✅ Хорошо |
| **Vanishing gradient** | ❌ Проблема | ✅ Решена | ✅ Решена |
| **Когда использовать** | Короткие последовательности (< 10) | Длинные последовательности, нужна максимальная точность | Баланс скорости и точности |

**Рекомендация:**
- Начинайте с **GRU** (быстрее, проще)
- Если GRU не справляется → **LSTM**
- Vanilla RNN — только для обучения

---

### 1.6 Продвинутые техники

#### Bidirectional RNN

Обрабатывает последовательность **в обе стороны:**

$$\vec{h}_t = \text{RNN}(x_1, x_2, ..., x_t)$$

$$\overleftarrow{h}_t = \text{RNN}(x_T, x_{T-1}, ..., x_t)$$

$$h_t = [\vec{h}_t; \overleftarrow{h}_t]$$

**Когда полезно:**
- ✅ Классификация последовательностей (весь контекст доступен)
- ❌ НЕ для прогнозирования (нельзя знать будущее!)

---

#### Stacked (Multi-layer) RNN

Несколько слоев RNN друг над другом:

```
x → [LSTM₁] → h₁ → [LSTM₂] → h₂ → [LSTM₃] → h₃ → y
```

**Преимущества:**
- ✅ Более глубокая иерархия признаков
- ✅ Лучшая производительность на сложных задачах

**Недостатки:**
- ❌ Больше параметров → переобучение на малых данных
- ❌ Дольше обучается

**Рекомендация:** 1-2 слоя обычно достаточно.

---

#### Teacher Forcing

**При обучении:** используем **истинные** значения $y_{t-1}$ как вход для предсказания $y_t$  
**При inference:** используем **предсказанные** значения $\hat{y}_{t-1}$

**Проблема:** model mismatch (train vs test)  
**Решение:** Scheduled sampling (постепенно переходим на предсказанные значения)

---

## 💻 Часть 2: Практика - RNN на PyTorch

### 2.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Для временных рядов (если нужно сравнить)
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Настройки
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Библиотеки загружены")

### 2.2 Загрузка и подготовка данных

Используем тот же Airline Passengers датасет.

In [ ]:
# Загрузка датасета
try:
    from statsmodels.datasets import get_rdataset
    airline_data = get_rdataset('AirPassengers', 'datasets')
    df = airline_data.data
    df.columns = ['time', 'passengers']
except:
    url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
    df = pd.read_csv(url, parse_dates=['Month'], index_col='Month')
    df.columns = ['passengers']
    df.reset_index(inplace=True)
    df.columns = ['time', 'passengers']

df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

print(f"Датасет: {df.shape[0]} наблюдений")
print(f"Период: {df.index.min()} - {df.index.max()}")

# Визуализация
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['passengers'], linewidth=2)
plt.title('Airline Passengers (1949-1960)', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Passengers')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

df.head()

### 2.3 Подготовка данных для RNN: Sliding Window

**Задача:** Создать последовательности (X) и целевые значения (y).

**Метод Sliding Window:**
- Используем последние `seq_length` точек для предсказания следующей
- Например: `[t-4, t-3, t-2, t-1]` → `t`

**Пример:**
```
Данные: [10, 20, 30, 40, 50, 60]
seq_length = 3

X:           y:
[10,20,30] → 40
[20,30,40] → 50
[30,40,50] → 60
```

In [ ]:
def create_sequences(data, seq_length):
    """
    Создает последовательности для RNN с помощью sliding window
    
    Args:
        data: массив numpy shape (n_samples,)
        seq_length: длина последовательности (look-back period)
    
    Returns:
        X: shape (n_sequences, seq_length, 1)
        y: shape (n_sequences, 1)
    """
    X, y = [], []
    
    for i in range(len(data) - seq_length):
        # Последовательность длины seq_length
        sequence = data[i:i + seq_length]
        # Следующее значение - целевое
        target = data[i + seq_length]
        
        X.append(sequence)
        y.append(target)
    
    X = np.array(X)
    y = np.array(y)
    
    # Добавляем размерность признаков (для univariate = 1)
    X = X.reshape(-1, seq_length, 1)
    y = y.reshape(-1, 1)
    
    return X, y

# Тест функции
test_data = np.array([10, 20, 30, 40, 50, 60])
test_X, test_y = create_sequences(test_data, seq_length=3)

print("Пример sliding window:")
print("Original data:", test_data)
print("\nSequences (X) and targets (y):")
for i in range(len(test_X)):
    print(f"X: {test_X[i].flatten()} → y: {test_y[i][0]}")

print(f"\nShape: X={test_X.shape}, y={test_y.shape}")

### 2.4 Нормализация и Train/Test Split

In [ ]:
# Извлекаем значения
data = df['passengers'].values.astype(float)

# Нормализация (важно для нейронных сетей!)
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data.reshape(-1, 1)).flatten()

print(f"Original range: [{data.min():.1f}, {data.max():.1f}]")
print(f"Normalized range: [{data_normalized.min():.3f}, {data_normalized.max():.3f}]")

# Параметры
SEQ_LENGTH = 12  # используем год данных для предсказания следующего месяца
TRAIN_SIZE = 0.8

# Train/Test split (по времени!)
train_size = int(len(data_normalized) * TRAIN_SIZE)
train_data = data_normalized[:train_size]
test_data = data_normalized[train_size - SEQ_LENGTH:]  # включаем SEQ_LENGTH для первой последовательности

print(f"\nTrain size: {len(train_data)} ({TRAIN_SIZE*100:.0f}%)")
print(f"Test size: {len(test_data) - SEQ_LENGTH} (фактические предсказания)")

# Создаем последовательности
X_train, y_train = create_sequences(train_data, SEQ_LENGTH)
X_test, y_test = create_sequences(test_data, SEQ_LENGTH)

print(f"\nTrain sequences: X={X_train.shape}, y={y_train.shape}")
print(f"Test sequences: X={X_test.shape}, y={y_test.shape}")

# Преобразуем в PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# DataLoader
BATCH_SIZE = 16

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)  # НЕ shuffle для TS!

print(f"\nBatch size: {BATCH_SIZE}")
print(f"Batches per epoch: {len(train_loader)}")

### 2.5 Модель 1: Vanilla RNN

Простая рекуррентная сеть для базового сравнения.

In [ ]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, output_size=1):
        super(VanillaRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # RNN layer
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True  # (batch, seq, feature)
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Инициализация hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # RNN forward pass
        # out: (batch, seq, hidden_size)
        # h_n: (num_layers, batch, hidden_size)
        out, h_n = self.rnn(x, h0)
        
        # Берем последний выход
        out = out[:, -1, :]
        
        # Fully connected
        out = self.fc(out)
        
        return out

# Создание модели
rnn_model = VanillaRNN(
    input_size=1,
    hidden_size=64,
    num_layers=1,
    output_size=1
).to(device)

print("Vanilla RNN Architecture:")
print(rnn_model)
print(f"\nTotal parameters: {sum(p.numel() for p in rnn_model.parameters())}")

### 2.6 Модель 2: LSTM

Long Short-Term Memory для долгосрочных зависимостей.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0  # dropout между слоями
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, output_size)
        )
    
    def forward(self, x):
        # Инициализация hidden и cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward pass
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        
        # Берем последний выход
        out = out[:, -1, :]
        
        # Fully connected
        out = self.fc(out)
        
        return out

# Создание модели
lstm_model = LSTMModel(
    input_size=1,
    hidden_size=64,
    num_layers=2,
    output_size=1,
    dropout=0.2
).to(device)

print("LSTM Architecture:")
print(lstm_model)
print(f"\nTotal parameters: {sum(p.numel() for p in lstm_model.parameters())}")

### 2.7 Модель 3: GRU

Gated Recurrent Unit - баланс между RNN и LSTM.

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(GRUModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # GRU layer
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, output_size)
        )
    
    def forward(self, x):
        # Инициализация hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # GRU forward pass
        out, h_n = self.gru(x, h0)
        
        # Берем последний выход
        out = out[:, -1, :]
        
        # Fully connected
        out = self.fc(out)
        
        return out

# Создание модели
gru_model = GRUModel(
    input_size=1,
    hidden_size=64,
    num_layers=2,
    output_size=1,
    dropout=0.2
).to(device)

print("GRU Architecture:")
print(gru_model)
print(f"\nTotal parameters: {sum(p.numel() for p in gru_model.parameters())}")

### 2.8 Функции обучения и оценки

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=100, patience=10):
    """
    Обучение RNN модели с early stopping
    """
    history = {'train_loss': []}
    best_loss = float('inf')
    patience_counter = 0
    
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        
        for X_batch, y_batch in train_loader:
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping (предотвращает exploding gradients)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        history['train_loss'].append(avg_loss)
        
        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    return history

def evaluate_model(model, X, y, scaler):
    """
    Оценка модели и денормализация предсказаний
    """
    model.eval()
    
    with torch.no_grad():
        predictions = model(X).cpu().numpy()
    
    # Денормализация
    y_true = scaler.inverse_transform(y.cpu().numpy())
    y_pred = scaler.inverse_transform(predictions)
    
    # Метрики
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    return y_pred, y_true, rmse, mae

print("✅ Функции обучения готовы")

## 🏋️ Часть 3: Обучение и сравнение моделей

### 3.1 Обучение всех моделей

In [ ]:
# Гиперпараметры
NUM_EPOCHS = 200
LEARNING_RATE = 0.001
PATIENCE = 15

# Loss и optimizer одинаковые для всех
criterion = nn.MSELoss()

# Словарь для хранения результатов
results = {}

print("="*60)
print("Training Configuration")
print("="*60)
print(f"Epochs: {NUM_EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Sequence Length: {SEQ_LENGTH}")
print(f"Early Stopping Patience: {PATIENCE}")
print("="*60)

#### 3.1.1 Обучение Vanilla RNN

In [ ]:
print("\n" + "="*60)
print("Training Vanilla RNN")
print("="*60)

rnn_optimizer = optim.Adam(rnn_model.parameters(), lr=LEARNING_RATE)
rnn_history = train_model(rnn_model, train_loader, criterion, rnn_optimizer, 
                          NUM_EPOCHS, PATIENCE)

# Evaluation
rnn_pred, rnn_true, rnn_rmse, rnn_mae = evaluate_model(rnn_model, X_test_tensor, 
                                                        y_test_tensor, scaler)

results['Vanilla RNN'] = {
    'model': rnn_model,
    'history': rnn_history,
    'predictions': rnn_pred,
    'rmse': rnn_rmse,
    'mae': rnn_mae
}

print(f"\n✅ Vanilla RNN trained")
print(f"   RMSE: {rnn_rmse:.2f}")
print(f"   MAE: {rnn_mae:.2f}")

#### 3.1.2 Обучение LSTM

In [ ]:
print("\n" + "="*60)
print("Training LSTM")
print("="*60)

lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)
lstm_history = train_model(lstm_model, train_loader, criterion, lstm_optimizer, 
                           NUM_EPOCHS, PATIENCE)

# Evaluation
lstm_pred, lstm_true, lstm_rmse, lstm_mae = evaluate_model(lstm_model, X_test_tensor, 
                                                           y_test_tensor, scaler)

results['LSTM'] = {
    'model': lstm_model,
    'history': lstm_history,
    'predictions': lstm_pred,
    'rmse': lstm_rmse,
    'mae': lstm_mae
}

print(f"\n✅ LSTM trained")
print(f"   RMSE: {lstm_rmse:.2f}")
print(f"   MAE: {lstm_mae:.2f}")

#### 3.1.3 Обучение GRU

In [ ]:
print("\n" + "="*60)
print("Training GRU")
print("="*60)

gru_optimizer = optim.Adam(gru_model.parameters(), lr=LEARNING_RATE)
gru_history = train_model(gru_model, train_loader, criterion, gru_optimizer, 
                          NUM_EPOCHS, PATIENCE)

# Evaluation
gru_pred, gru_true, gru_rmse, gru_mae = evaluate_model(gru_model, X_test_tensor, 
                                                       y_test_tensor, scaler)

results['GRU'] = {
    'model': gru_model,
    'history': gru_history,
    'predictions': gru_pred,
    'rmse': gru_rmse,
    'mae': gru_mae
}

print(f"\n✅ GRU trained")
print(f"   RMSE: {gru_rmse:.2f}")
print(f"   MAE: {gru_mae:.2f}")

### 3.2 Сравнение с классической SARIMA

Добавим SARIMA из предыдущего ноутбука для честного сравнения.

In [ ]:
print("\n" + "="*60)
print("Training SARIMA for comparison")
print("="*60)

# Используем оригинальные (ненормализованные) данные
train_size_orig = int(len(data) * TRAIN_SIZE)
train_orig = data[:train_size_orig]
test_orig = data[train_size_orig:]

# SARIMA(1, 1, 1)(1, 1, 1, 12)
sarima_model = SARIMAX(
    train_orig,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 12),
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_fitted = sarima_model.fit(disp=False)
sarima_forecast = sarima_fitted.forecast(steps=len(test_orig))

# Метрики
sarima_rmse = np.sqrt(mean_squared_error(test_orig, sarima_forecast))
sarima_mae = mean_absolute_error(test_orig, sarima_forecast)

results['SARIMA'] = {
    'predictions': sarima_forecast,
    'rmse': sarima_rmse,
    'mae': sarima_mae
}

print(f"\n✅ SARIMA trained")
print(f"   RMSE: {sarima_rmse:.2f}")
print(f"   MAE: {sarima_mae:.2f}")

### 3.3 Визуализация результатов

In [ ]:
# Training curves
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(results['Vanilla RNN']['history']['train_loss'], label='Vanilla RNN', linewidth=2)
ax.plot(results['LSTM']['history']['train_loss'], label='LSTM', linewidth=2)
ax.plot(results['GRU']['history']['train_loss'], label='GRU', linewidth=2)

ax.set_title('Training Loss Curves', fontsize=16, fontweight='bold')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss (MSE)', fontsize=12)
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 RNN модели обучаются стабильно, LSTM/GRU сходятся быстрее Vanilla RNN")

In [ ]:
# Predictions visualization
# Получаем индексы для test данных
test_indices = df.index[train_size_orig:]

plt.figure(figsize=(14, 7))

# Train данные (контекст)
plt.plot(df.index[:train_size_orig], data[:train_size_orig], 
         label='Train', linewidth=2, alpha=0.5, color='gray')

# Истинные test значения
plt.plot(test_indices, test_orig, label='Test (True)', 
         linewidth=3, color='black')

# Предсказания RNN моделей
plt.plot(test_indices, results['Vanilla RNN']['predictions'], 
         label=f"Vanilla RNN (RMSE={rnn_rmse:.1f})", linewidth=2, linestyle='--')
plt.plot(test_indices, results['LSTM']['predictions'], 
         label=f"LSTM (RMSE={lstm_rmse:.1f})", linewidth=2, linestyle='--')
plt.plot(test_indices, results['GRU']['predictions'], 
         label=f"GRU (RMSE={gru_rmse:.1f})", linewidth=2, linestyle='--')

# SARIMA
plt.plot(test_indices, results['SARIMA']['predictions'], 
         label=f"SARIMA (RMSE={sarima_rmse:.1f})", linewidth=2, linestyle=':')

plt.axvline(df.index[train_size_orig], color='red', linestyle='--', 
            alpha=0.3, linewidth=2, label='Train/Test Split')

plt.title('Model Comparison: RNN vs LSTM vs GRU vs SARIMA', 
          fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Number of Passengers', fontsize=12)
plt.legend(loc='upper left', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 3.4 Сравнительная таблица метрик

In [ ]:
# Создание таблицы сравнения
comparison_df = pd.DataFrame({
    'Model': ['Vanilla RNN', 'LSTM', 'GRU', 'SARIMA'],
    'RMSE': [
        results['Vanilla RNN']['rmse'],
        results['LSTM']['rmse'],
        results['GRU']['rmse'],
        results['SARIMA']['rmse']
    ],
    'MAE': [
        results['Vanilla RNN']['mae'],
        results['LSTM']['mae'],
        results['GRU']['mae'],
        results['SARIMA']['mae']
    ],
    'Parameters': [
        sum(p.numel() for p in rnn_model.parameters()),
        sum(p.numel() for p in lstm_model.parameters()),
        sum(p.numel() for p in gru_model.parameters()),
        'N/A'
    ],
    'Type': ['Deep Learning', 'Deep Learning', 'Deep Learning', 'Classical']
})

# Сортируем по RMSE
comparison_df = comparison_df.sort_values('RMSE')

print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Визуализация метрик
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
comparison_df.plot(x='Model', y='RMSE', kind='bar', ax=axes[0], legend=False, color='steelblue')
axes[0].set_title('RMSE Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)

# MAE comparison
comparison_df.plot(x='Model', y='MAE', kind='bar', ax=axes[1], legend=False, color='coral')
axes[1].set_title('MAE Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🎓 Выводы и рекомендации

### 📊 Результаты на Airline Passengers

**Ожидаемая ситуация для этого датасета:**

1. **SARIMA чаще всего лучше** RNN-моделей
   - ✅ Маленький датасет (144 точки)
   - ✅ Четкая сезонность (SARIMA заточена под это)
   - ✅ Univariate (одна переменная)

2. **LSTM/GRU ≈ сопоставимы** с SARIMA или чуть хуже
   - ⚠️ Мало данных для глубокого обучения
   - ✅ Но захватывают нелинейные паттерны

3. **Vanilla RNN хуже всех**
   - ❌ Проблема vanishing gradient
   - ❌ Не справляется с долгосрочными зависимостями

---

### 🎯 Когда использовать RNN/LSTM/GRU?

**Deep Learning лучше классических методов когда:**

| Характеристика | Classical (ARIMA/SARIMA) | Deep Learning (LSTM/GRU) |
|----------------|-------------------------|-------------------------|
| **Размер данных** | < 1000 точек | > 1000 точек (чем больше, тем лучше) |
| **Количество признаков** | Univariate (1 переменная) | **Multivariate (много переменных)** |
| **Паттерны** | Линейные, сезонные | **Нелинейные, сложные** |
| **Скорость обучения** | **Быстро (секунды)** | Медленно (минуты/часы) |
| **Интерпретируемость** | **Высокая** | Низкая (черный ящик) |
| **Автоматизация** | Средняя (подбор p, d, q) | **Высокая (end-to-end)** |

**RNN/LSTM/GRU побеждают когда:**
- ✅ **Multivariate time series** (много взаимосвязанных переменных)
- ✅ **Большие датасеты** (тысячи/миллионы точек)
- ✅ **Нелинейные зависимости** (сложные паттерны)
- ✅ **Нестандартная сезонность** (несколько периодов, нерегулярная)
- ✅ **Много внешних факторов** (легко добавить как признаки)

**Примеры задач для RNN:**
- 🏭 **IoT сенсоры:** температура, давление, вибрация одновременно
- 📈 **Финансы:** цена + объем + индикаторы + новости
- ⚡ **Энергетика:** потребление + погода + день недели + праздники
- 🏥 **Медицина:** мониторинг пациентов (пульс, давление, температура)

---

### 🔧 Практические рекомендации

**Рабочий алгоритм выбора модели:**

```
1. Начните с ARIMA/SARIMA (baseline)
   ↓
2. Если результат неудовлетворителен:
   - Маленький датасет (< 1k) → попробуйте Prophet
   - Большой датасет (> 1k) → попробуйте LSTM/GRU
   ↓
3. Для multivariate:
   - VAR (Vector AutoRegression) - классика
   - LSTM/GRU - deep learning
   ↓
4. Ансамбль:
   - SARIMA + LSTM часто лучше каждого по отдельности
```

**Тюнинг RNN моделей:**

1. **Sequence length (look-back period)**
   - Для сезонности: минимум 1 период (12 для месячных данных)
   - Слишком большой → переобучение
   - Слишком маленький → недостаточно контекста

2. **Hidden size**
   - Малые данные: 32-64
   - Средние: 64-128
   - Большие: 128-512

3. **Number of layers**
   - 1-2 слоя обычно достаточно
   - Больше → переобучение на малых данных

4. **Dropout**
   - 0.1-0.3 для регуляризации
   - Критично на малых датасетах

5. **Learning rate**
   - Начните с 0.001
   - Используйте scheduler (ReduceLROnPlateau)

---

### 🚀 Следующие шаги

**Продвинутые техники (Phase 3, Step 3):**
- **Attention механизм** для RNN
- **Seq2Seq** модели для multi-step forecasting
- **Encoder-Decoder** архитектура
- **Transformer** для временных рядов (Phase 4)

**Практические улучшения:**
- **Ансамбли:** SARIMA + LSTM
- **Multivariate:** добавить внешние признаки
- **Transfer learning:** предобучение на других TS
- **Multi-step forecasting:** предсказание на несколько шагов

---

### 📚 Дополнительные материалы

**Теория:**
- ["Understanding LSTM Networks" (Colah's Blog)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- ["The Unreasonable Effectiveness of RNN" (Karpathy)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

**Практика:**
- [PyTorch RNN Tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)
- [Time Series with LSTM](https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/)

---

**Phase 3, Step 2 COMPLETE!** ✅  
**Next:** Advanced RNN - Attention & Seq2Seq (Step 3)